<a href="https://colab.research.google.com/github/stavco9/datastreaming-final-project/blob/main/3.1-Deezy-Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install DeezyMatch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 

In [3]:
import os
import tqdm
from DeezyMatch import train as dm_train, plot_log, finetune as dm_finetune, inference as dm_inference, combine_vecs, candidate_ranker, candidate_ranker_init


/usr/local/lib/python3.10/dist-packages/DeezyMatch/rnn_networks.py:30: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset"

Mounted at /content/drive
/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset


In [5]:
# generate vectors for candidates (specified in dataset_path)
# using a model stored at pretrained_model_path and pretrained_vocab_path
dm_inference(input_file_path="./train_conf.yaml",
             dataset_path="./candidates/dataset_candidates.txt",
             pretrained_model_path="./models/wikigaz_en_model/wikigaz_en_model.model",
             pretrained_vocab_path="./models/wikigaz_en_model/wikigaz_en_model.vocab",
             inference_mode="vect",
             scenario="candidates/test")

2024-03-22 21:37:02 5dc4d4d450da [INFO] read input file: ./train_conf.yaml
2024-03-22 21:37:02 5dc4d4d450da [INFO] pytorch will use: cuda
2024-03-22 21:37:06 5dc4d4d450da [INFO] read CSV file: ./candidates/dataset_candidates.txt
2024-03-22 21:37:09 5dc4d4d450da [INFO] number of labels, True: 387264 and False: 0
2024-03-22 21:37:22 5dc4d4d450da [INFO] skipping 0 lines


2024-03-22 21:37:25 5dc4d4d450da [INFO] save test-data-class: /content/drive/MyDrive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset/candidates/test/dataframe.df


  0%|          | 0/1513 [00:00<?, ?it/s]

--- 101.39423656463623 seconds ---


In [6]:
# combine vectors stored in candidates/test and save them in combined/candidates_test
combine_vecs(rnn_passes=['fwd', 'bwd'],
             input_scenario='candidates/test',
             output_scenario='combined/candidates_test',
             print_every=10)

2024-03-22 21:39:16 5dc4d4d450da [INFO] read input file: candidates/test/train_conf.yaml
2024-03-22 21:39:17 5dc4d4d450da [INFO] pytorch will use: cuda


-- Combine vectors
Reading vectors from candidates/test/embeddings/rnn_fwd*
0000000 candidates/test/embeddings/rnn_fwd_0
0000010 candidates/test/embeddings/rnn_fwd_10
0000020 candidates/test/embeddings/rnn_fwd_20
0000030 candidates/test/embeddings/rnn_fwd_30
0000040 candidates/test/embeddings/rnn_fwd_40
0000050 candidates/test/embeddings/rnn_fwd_50
0000060 candidates/test/embeddings/rnn_fwd_60
0000070 candidates/test/embeddings/rnn_fwd_70
0000080 candidates/test/embeddings/rnn_fwd_80
0000090 candidates/test/embeddings/rnn_fwd_90
0000100 candidates/test/embeddings/rnn_fwd_100
0000110 candidates/test/embeddings/rnn_fwd_110
0000120 candidates/test/embeddings/rnn_fwd_120
0000130 candidates/test/embeddings/rnn_fwd_130
0000140 candidates/test/embeddings/rnn_fwd_140
0000150 candidates/test/embeddings/rnn_fwd_150
0000160 candidates/test/embed

In [19]:
some_sentance_with_error = "I wanna appel pleases or a bannana with a paer and a lemone, \
  or a wartermelon if possibe. I hope i don't have an errof in Londaon and in Jerusealm"

# Ranking on-the-fly
# find candidates from candidate_scenario
# for queries specified by the `query` argument
ranker = \
    candidate_ranker_init(candidate_scenario="./combined/candidates_test",
                     query=[x.replace("-", " ") for x in some_sentance_with_error.split()],
                     ranking_metric="faiss",
                     selection_threshold=10.,
                     num_candidates=2,
                     search_size=100,
                     output_path="ranker_results/test_candidates_deezymatch_on_the_fly",
                     pretrained_model_path="./models/wikigaz_en_model/wikigaz_en_model.model",
                     pretrained_vocab_path="./models/wikigaz_en_model/wikigaz_en_model.vocab",
                     number_test_rows=len(some_sentance_with_error))

ranker.rank()
ranker_pd = ranker.output

2024-03-22 23:19:30 f07e22b38c84 [INFO] read input file: ./combined/candidates_test/train_conf.yaml
2024-03-22 23:19:30 f07e22b38c84 [INFO] pytorch will use: cuda
2024-03-22 23:19:30 f07e22b38c84 [INFO] use a dataframe in test_tokenize.
2024-03-22 23:19:30 f07e22b38c84 [INFO] number of labels, True: 0 and False: 30
2024-03-22 23:19:30 f07e22b38c84 [INFO] skipping 0 lines


2024-03-22 23:19:30 f07e22b38c84 [INFO] save test-data-class: tmp_843b8aaf-9fb3-4aa8-abca-c63552f50933/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_843b8aaf-9fb3-4aa8-abca-c63552f50933/query/embeddings/rnn_fwd*
0000000 tmp_843b8aaf-9fb3-4aa8-abca-c63552f50933/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_843b8aaf-9fb3-4aa8-abca-c63552f50933/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_843b8aaf-9fb3-4aa8-abca-c63552f50933/query/embeddings/rnn_bwd*
0000000 tmp_843b8aaf-9fb3-4aa8-abca-c63552f50933/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_843b8aaf-9fb3-4aa8-abca-c63552f50933/query/embeddings/rnn_indxs_0
--- 603.721617937088 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/30 -- Number of found candidates so far: 2, searched: 100
=========== Start the search for 1 wanna
ID: 2/30 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 2 appel
ID: 3/30 -- Number of found candidates so far: 17, searched: 100
=========== Start the search for 3 pleases
ID: 4/30 

In [20]:
for index, row in ranker_pd.iterrows():
    print(f"Possible matches for word {row['query']}:")
    for _idx, _key in enumerate(list(row['faiss_distance'].keys())):
      print(f"{_idx + 1}) {_key} with distance of {row['faiss_distance'][_key]}")

Possible matches for word I:
1) I with distance of 0.0
2) i with distance of 0.0
Possible matches for word wanna:
1) Hanna with distance of 7.2827
2) with with distance of 8.135
Possible matches for word appel:
1) appealed with distance of 6.1974
2) Capel with distance of 6.3239
Possible matches for word pleases:
1) settles with distance of 5.0393
2) Laes with distance of 5.0803
Possible matches for word or:
1) or with distance of 0.0
2) Or with distance of 0.0
Possible matches for word a:
1) a with distance of 0.0
2) A with distance of 0.0
Possible matches for word bannana:
1) Nana with distance of 5.6122
2) Gabbana with distance of 6.5312
Possible matches for word with:
1) with with distance of 0.0
2) With with distance of 0.0
Possible matches for word a:
1) a with distance of 0.0
2) A with distance of 0.0
Possible matches for word paer:
1) Parker with distance of 3.4475
2) parameter with distance of 4.5089
Possible matches for word and:
1) and with distance of 0.0
2) And with distan